In [1]:
import os
os.environ['CUDA_VISIBLE_DEVICES'] = "1"
import warnings
warnings.filterwarnings("ignore") 
import subprocess
from utils import load_jsonl, extract_code_block, extract_obj, change_variable_types
import numpy as np
from vllm import LLM, SamplingParams        
from transformers import AutoTokenizer                                      
from langchain.prompts import PromptTemplate






INFO 06-18 17:34:33 [__init__.py:244] Automatically detected platform cuda.


2025-06-18 17:34:34,690	INFO util.py:154 -- Missing packages: ['ipywidgets']. Run `pip install -U ipywidgets`, then restart the notebook server for rich notebook output.


In [2]:
# load checkpoints and tokenizer

model_path = '/DATA/disk1/cml/sft_copt/Qwen3_8B_sft'
tensor_parallel_size = 1
solver_name = 'gurobi'
print("Loading model", model_path)
model = LLM(
    model=model_path,
    tensor_parallel_size=tensor_parallel_size,
    trust_remote_code=True
)
print("Model initialized.")
tokenizer = AutoTokenizer.from_pretrained(model_path, trust_remote_code=True)



Loading model /DATA/disk1/cml/sft_copt/Qwen3_8B_sft
INFO 06-18 14:45:25 [config.py:823] This model supports multiple tasks: {'embed', 'score', 'classify', 'generate', 'reward'}. Defaulting to 'generate'.
INFO 06-18 14:45:25 [config.py:2195] Chunked prefill is enabled with max_num_batched_tokens=16384.
WARNING 06-18 14:45:26 [utils.py:2597] We must use the `spawn` multiprocessing start method. Overriding VLLM_WORKER_MULTIPROC_METHOD to 'spawn'. See https://docs.vllm.ai/en/latest/usage/troubleshooting.html#python-multiprocessing for more information. Reason: CUDA is initialized
WARNING 06-18 14:45:27 [env_override.py:17] NCCL_CUMEM_ENABLE is set to 0, skipping override. This may increase memory overhead with cudagraph+allreduce: https://github.com/NVIDIA/nccl/issues/1234
INFO 06-18 14:45:30 [__init__.py:244] Automatically detected platform cuda.
INFO 06-18 14:45:32 [core.py:455] Waiting for init message from front-end.
INFO 06-18 14:45:32 [core.py:70] Initializing a V1 LLM engine (v0.9.1

Loading safetensors checkpoint shards:   0% Completed | 0/5 [00:00<?, ?it/s]
Loading safetensors checkpoint shards:  20% Completed | 1/5 [00:00<00:02,  1.67it/s]
Loading safetensors checkpoint shards:  40% Completed | 2/5 [00:00<00:01,  2.26it/s]
Loading safetensors checkpoint shards:  60% Completed | 3/5 [00:01<00:01,  1.90it/s]
Loading safetensors checkpoint shards:  80% Completed | 4/5 [00:02<00:00,  1.65it/s]
Loading safetensors checkpoint shards: 100% Completed | 5/5 [00:02<00:00,  1.65it/s]
Loading safetensors checkpoint shards: 100% Completed | 5/5 [00:02<00:00,  1.73it/s]



INFO 06-18 14:45:36 [default_loader.py:272] Loading weights took 3.00 seconds
INFO 06-18 14:45:37 [gpu_model_runner.py:1624] Model loading took 15.2683 GiB and 3.140523 seconds
INFO 06-18 14:45:43 [backends.py:462] Using cache directory: /DATA/disk1/cml/config/cache/vllm/torch_compile_cache/6bfa125916/rank_0_0 for vLLM's torch.compile
INFO 06-18 14:45:43 [backends.py:472] Dynamo bytecode transform time: 6.08 s
INFO 06-18 14:45:48 [backends.py:135] Directly load the compiled graph(s) for shape None from the cache, took 4.300 s
INFO 06-18 14:45:48 [monitor.py:34] torch.compile takes 6.08 s in total
INFO 06-18 14:45:49 [gpu_worker.py:227] Available KV cache memory: 50.29 GiB
INFO 06-18 14:45:50 [kv_cache_utils.py:715] GPU KV cache size: 366,192 tokens
INFO 06-18 14:45:50 [kv_cache_utils.py:719] Maximum concurrency for 40,960 tokens per request: 8.94x
INFO 06-18 14:46:08 [gpu_model_runner.py:2048] Graph capturing finished in 18 secs, took 0.74 GiB
INFO 06-18 14:46:08 [core.py:171] init eng

In [2]:
# load checkpoints and tokenizer

model_path = '/DATA/disk1/cml/sft_copt/Qwen3_8B_sft'
tensor_parallel_size = 1
solver_name = 'copt'
print("Loading model", model_path)
model = LLM(
    model=model_path,
    tensor_parallel_size=tensor_parallel_size,
    trust_remote_code=True
)
print("Model initialized.")
tokenizer = AutoTokenizer.from_pretrained(model_path, trust_remote_code=True)



Loading model /DATA/disk1/cml/sft_copt/Qwen3_8B_sft
INFO 06-18 17:34:46 [config.py:823] This model supports multiple tasks: {'classify', 'score', 'generate', 'embed', 'reward'}. Defaulting to 'generate'.
INFO 06-18 17:34:46 [config.py:2195] Chunked prefill is enabled with max_num_batched_tokens=16384.
WARNING 06-18 17:34:48 [utils.py:2597] We must use the `spawn` multiprocessing start method. Overriding VLLM_WORKER_MULTIPROC_METHOD to 'spawn'. See https://docs.vllm.ai/en/latest/usage/troubleshooting.html#python-multiprocessing for more information. Reason: CUDA is initialized
WARNING 06-18 17:34:49 [env_override.py:17] NCCL_CUMEM_ENABLE is set to 0, skipping override. This may increase memory overhead with cudagraph+allreduce: https://github.com/NVIDIA/nccl/issues/1234
INFO 06-18 17:34:52 [__init__.py:244] Automatically detected platform cuda.
INFO 06-18 17:34:55 [core.py:455] Waiting for init message from front-end.
INFO 06-18 17:34:55 [core.py:70] Initializing a V1 LLM engine (v0.9.1

Loading safetensors checkpoint shards:   0% Completed | 0/5 [00:00<?, ?it/s]
Loading safetensors checkpoint shards:  20% Completed | 1/5 [00:00<00:02,  1.36it/s]
Loading safetensors checkpoint shards:  40% Completed | 2/5 [00:01<00:01,  2.04it/s]
Loading safetensors checkpoint shards:  60% Completed | 3/5 [00:01<00:01,  1.69it/s]
Loading safetensors checkpoint shards:  80% Completed | 4/5 [00:02<00:00,  1.53it/s]
Loading safetensors checkpoint shards: 100% Completed | 5/5 [00:03<00:00,  1.58it/s]
Loading safetensors checkpoint shards: 100% Completed | 5/5 [00:03<00:00,  1.61it/s]



INFO 06-18 17:34:59 [default_loader.py:272] Loading weights took 3.16 seconds
INFO 06-18 17:34:59 [gpu_model_runner.py:1624] Model loading took 15.2683 GiB and 3.297829 seconds
INFO 06-18 17:35:06 [backends.py:462] Using cache directory: /DATA/disk1/cml/config/cache/vllm/torch_compile_cache/6bfa125916/rank_0_0 for vLLM's torch.compile
INFO 06-18 17:35:06 [backends.py:472] Dynamo bytecode transform time: 6.13 s
INFO 06-18 17:35:10 [backends.py:135] Directly load the compiled graph(s) for shape None from the cache, took 4.283 s
INFO 06-18 17:35:11 [monitor.py:34] torch.compile takes 6.13 s in total
INFO 06-18 17:35:12 [gpu_worker.py:227] Available KV cache memory: 50.29 GiB
INFO 06-18 17:35:12 [kv_cache_utils.py:715] GPU KV cache size: 366,192 tokens
INFO 06-18 17:35:12 [kv_cache_utils.py:719] Maximum concurrency for 40,960 tokens per request: 8.94x
INFO 06-18 17:35:31 [gpu_model_runner.py:2048] Graph capturing finished in 19 secs, took 0.74 GiB
INFO 06-18 17:35:31 [core.py:171] init eng

In [ ]:
# load prompt template and functions for Gurobi generation
from rule_prompt_utils import system_prompt_temp
zeroshot_prompt_system = PromptTemplate.from_template(system_prompt_temp['system'])
zeroshot_prompt_user = PromptTemplate.from_template(system_prompt_temp['user'])
def mp_worker(item):
    prompt = [
        {
            "role": "system",
            "content": zeroshot_prompt_system.format(question=item['en_question']).strip()
        },
        {
            "role": "user",
            "content": zeroshot_prompt_user.format(question=item['en_question']).strip()
        }
    ]
    text = tokenizer.apply_chat_template(prompt, tokenize=False, add_generation_prompt=True)
    return text

def generate_with_model(model, prompt, sampling_params):   
    response = model.generate(prompt, sampling_params) 
    result_text = [g.outputs[0].text for g in response]
    return result_text

In [3]:
# load prompt template and functions for Copt generation
from rule_prompt_utils_copt import system_prompt_temp
zeroshot_prompt_system = PromptTemplate.from_template(system_prompt_temp['system'])
zeroshot_prompt_user = PromptTemplate.from_template(system_prompt_temp['user'])
def mp_worker(item):
    prompt = [
        {
            "role": "system",
            "content": zeroshot_prompt_system.format(question=item['en_question']).strip()
        },
        {
            "role": "user",
            "content": zeroshot_prompt_user.format(question=item['en_question']).strip()
        }
    ]
    text = tokenizer.apply_chat_template(prompt, tokenize=False, add_generation_prompt=True)
    return text

def generate_with_model(model, prompt, sampling_params):   
    response = model.generate(prompt, sampling_params) 
    result_text = [g.outputs[0].text for g in response]
    return result_text

In [4]:
# Load decode strategy
topk = 1
max_tokens = 8192
repetition_penalty = 1.02 # To avoid the occasional occurrence of repeated tokens
stop_tokens = ["</s>"]

# top-p strategy
sampling_params = SamplingParams(
    n=topk,
    temperature=0.5,
    top_p=0.9,
    max_tokens=max_tokens,
    stop=stop_tokens,
    repetition_penalty=repetition_penalty
)


In [ ]:
# check the pass@1 accuracy for Gurobi
def check_result(result_str, item, solver_name='gurobi'):
    sub_answer = item['en_answer']
    # Convert sub_answer to float or None
    sub_answer = None if sub_answer == "No Best Solution" or "-9999" in str(sub_answer) else float(sub_answer)
    
    # Extract code snippet
    code_snippet = extract_code_block(result_str, solver_name)
    #print(code_snippet)
    if not code_snippet:
        return 2
    
    # Run code snippet
    try:
        result = subprocess.run(['python3', '-c', code_snippet], capture_output=True, text=True, timeout=100)
    except subprocess.TimeoutExpired:
        return 1 if sub_answer is None else 0
    
    # Check if execution failed
    if result.returncode != 0:
        return 3
    
    # Extract solver result
    solver_result = extract_obj(result.stdout)
    
    # check the first time
    if solver_result is not None and sub_answer is not None and np.abs(solver_result - sub_answer) / (np.abs(sub_answer) + 1) <= 1e-6:
        return 1
    # Handle infeasible case or numerical mismatch since we ignore the variable types error
    if 'nfeasible' in result.stdout or (solver_result is not None and sub_answer is not None and np.abs(solver_result - sub_answer) / (np.abs(sub_answer) + 1) > 1e-6):
        # Try re-running with modified variables: we ignore the variable types error
        result_str = change_variable_types(result_str) # change the type of variables
        if result_str:
            try:
                code_snippet = extract_code_block(result_str, solver_name)
                result = subprocess.run(['python3', '-c', code_snippet], capture_output=True, text=True, timeout=100)
                if result.returncode == 0:
                    new_result = extract_obj(result.stdout)
                    if 'nfeasible' not in result.stdout: # infeasible and Infeasible
                        if new_result is not None and sub_answer is not None and np.abs(new_result - sub_answer) / (np.abs(sub_answer) + 1) < 1e-6:
                            return 1
                        if new_result == sub_answer:
                            return 1
            except subprocess.TimeoutExpired:
                print("over_time")
    
    # Handle infeasible case after retry
    if 'nfeasible' in result.stdout:
        return 1 if sub_answer is None else 0
    
    # Final comparison
    if solver_result is not None and sub_answer is not None:
        return 1 if np.abs(solver_result - sub_answer) / (np.abs(sub_answer) + 1) < 1e-6 else 0
    return 1 if solver_result == sub_answer else 0

In [5]:
# check the pass@1 accuracy for Copt
def check_result(result_str, item, solver_name='copt'):
    sub_answer = item['en_answer']
    # Convert sub_answer to float or None
    sub_answer = None if sub_answer == "No Best Solution" or "-9999" in str(sub_answer) else float(sub_answer)
    
    # Extract code snippet
    code_snippet = extract_code_block(result_str, solver_name)
    with open('ouput_code.txt','a',encoding='utf-8') as f:
        f.write(f'{code_snippet}\n')
    #print(code_snippet)
    if not code_snippet:
        return 2
    
    # Run code snippet
    try:
        result = subprocess.run(['python3', '-c', code_snippet], capture_output=True, text=True, timeout=100)
    except subprocess.TimeoutExpired:
        return 1 if sub_answer is None else 0
    
    # Check if execution failed
    if result.returncode != 0:
        return 3
    
    # Extract solver result
    solver_result = extract_obj(result.stdout)
    
    # check the first time
    if solver_result is not None and sub_answer is not None and np.abs(solver_result - sub_answer) / (np.abs(sub_answer) + 1) <= 1e-6:
        return 1
    # Handle infeasible case or numerical mismatch since we ignore the variable types error
    if 'nfeasible' in result.stdout or (solver_result is not None and sub_answer is not None and np.abs(solver_result - sub_answer) / (np.abs(sub_answer) + 1) > 1e-6):
        # Try re-running with modified variables: we ignore the variable types error
        result_str = change_variable_types(result_str) # change the type of variables
        if result_str:
            try:
                code_snippet = extract_code_block(result_str, solver_name)
                result = subprocess.run(['python3', '-c', code_snippet], capture_output=True, text=True, timeout=100)
                if result.returncode == 0:
                    new_result = extract_obj(result.stdout)
                    if 'nfeasible' not in result.stdout: # infeasible and Infeasible
                        if new_result is not None and sub_answer is not None and np.abs(new_result - sub_answer) / (np.abs(sub_answer) + 1) < 1e-6:
                            return 1
                        if new_result == sub_answer:
                            return 1
            except subprocess.TimeoutExpired:
                print("over_time")
    
    # Handle infeasible case after retry
    if 'nfeasible' in result.stdout:
        return 1 if sub_answer is None else 0
    
    # Final comparison
    if solver_result is not None and sub_answer is not None:
        return 1 if np.abs(solver_result - sub_answer) / (np.abs(sub_answer) + 1) < 1e-6 else 0
    return 1 if solver_result == sub_answer else 0

In [ ]:
# if you want to check pass@1 accuracy, please run this cell
# Test the checkpoint
datapath = 'test_data'
testdataset = ['NL4OPT.jsonl', 'MAMO_EasyLP.json', 'MAMO_ComplexLP.json', 'IndustryOR_fixed.json', 'OptMATH_Bench_193.jsonl', 'OptMATH_Bench_166.jsonl','OptiBench.jsonl']
for filepath in testdataset:
    
    # loading data
    print('Loading data', filepath)
    test_data = load_jsonl(os.path.join(datapath, filepath))
    print('Finish Loading')
    
    # generation 
    
    prompt_list = []
    for item in test_data:
        prompt_list.append(mp_worker(item))
    result_strs = generate_with_model(model, prompt_list, sampling_params)
    snippet_package_cor = []
    score = []
    # check the pass@1 accuracy
    
    for result_str, item in zip(result_strs, test_data):
        snippet_package_cor.append(check_result(result_str, item, solver_name))
    result = np.bincount(snippet_package_cor)
    print(f'Numbers of test cases in dataset {filepath}: {sum(result)}')
    print(f'Numbers of pass@1 cases in dataset {filepath}: {result[1]}')
    print(f'pass@1 accuracy for dataset {filepath}: {result[1]}/{sum(result)} = {result[1] / sum(result)}')
    print('-------------------------------------------------------------------')
  

Loading data NL4OPT.jsonl
Finish Loading


Adding requests:   0%|          | 0/245 [00:00<?, ?it/s]

Processed prompts: 100%|██████████| 245/245 [03:11<00:00,  1.28it/s, est. speed input: 413.52 toks/s, output: 4260.03 toks/s]


Numbers of test cases in dataset NL4OPT.jsonl: 245
Numbers of pass@1 cases in dataset NL4OPT.jsonl: 207
pass@1 accuracy for dataset NL4OPT.jsonl: 207/245 = 0.8448979591836735
-------------------------------------------------------------------
Loading data MAMO_EasyLP.json
Finish Loading


Processed prompts: 100%|██████████| 652/652 [09:47<00:00,  1.11it/s, est. speed input: 467.48 toks/s, output: 3993.84 toks/s]


...

Numbers of test cases in dataset OptMATH_Bench_193.jsonl: 193
Numbers of pass@1 cases in dataset OptMATH_Bench_193.jsonl: 23
pass@1 accuracy for dataset OptMATH_Bench_193.jsonl: 23/193 = 0.11917098445595854
-------------------------------------------------------------------
Loading data OptMATH_Bench_166.jsonl
Finish Loading


Processed prompts: 100%|██████████| 166/166 [06:45<00:00,  2.44s/it, est. speed input: 484.73 toks/s, output: 2603.75 toks/s]


Numbers of test cases in dataset OptMATH_Bench_166.jsonl: 166
Numbers of pass@1 cases in dataset OptMATH_Bench_166.jsonl: 15
pass@1 accuracy for dataset OptMATH_Bench_166.jsonl: 15/166 = 0.09036144578313253
-------------------------------------------------------------------
Loading data OptiBench.jsonl
Finish Loading


Processed prompts: 100%|██████████| 605/605 [09:28<00:00,  1.06it/s, est. speed input: 384.29 toks/s, output: 3974.45 toks/s]


Numbers of test cases in dataset OptiBench.jsonl: 605
Numbers of pass@1 cases in dataset OptiBench.jsonl: 365
pass@1 accuracy for dataset OptiBench.jsonl: 365/605 = 0.6033057851239669
-------------------------------------------------------------------


In [6]:
# if you want to check pass@1 accuracy, please run this cell
# Test the checkpoint
datapath = 'test_data'
testdataset = ['NL4OPT.jsonl', 'MAMO_EasyLP.json', 'MAMO_ComplexLP.json', 'IndustryOR_fixed.json', 'OptMATH_Bench_193.jsonl', 'OptMATH_Bench_166.jsonl','OptiBench.jsonl']
for filepath in testdataset[3:4]:
    
    # loading data
    print('Loading data', filepath)
    test_data = load_jsonl(os.path.join(datapath, filepath))
    print('Finish Loading')
    
    # generation 
    
    prompt_list = []
    for item in test_data[:20]:
        prompt_list.append(mp_worker(item))
    result_strs = generate_with_model(model, prompt_list, sampling_params)
    snippet_package_cor = []
    score = []
    # check the pass@1 accuracy
    
    for result_str, item in zip(result_strs, test_data[:20]):
        snippet_package_cor.append(check_result(result_str, item, solver_name))
    result = np.bincount(snippet_package_cor)
    """print(f'Numbers of test cases in dataset {filepath}: {sum(result)}')
    print(f'Numbers of pass@1 cases in dataset {filepath}: {result[1]}')
    print(f'pass@1 accuracy for dataset {filepath}: {result[1]}/{sum(result)} = {result[1] / sum(result)}')
    print('-------------------------------------------------------------------')
  """

Loading data IndustryOR_fixed.json
Finish Loading


Processed prompts:   0%|          | 0/20 [00:00<?, ?it/s, est. speed input: 0.00 toks/s, output: 0.00 toks/s]

Processed prompts: 100%|██████████| 20/20 [01:27<00:00,  4.38s/it, est. speed input: 100.62 toks/s, output: 1197.47 toks/s]


UnboundLocalError: cannot access local variable 'pattern' where it is not associated with a value

In [7]:
# if you want to check pass@8 accuracy, please run this cell
# Test the checkpoint
datapath = 'test_data'
testdataset = ['NL4OPT.jsonl', 'MAMO_EasyLP.json', 'MAMO_ComplexLP.json', 'IndustryOR_fixed.json', 'OptMATH_Bench_193.jsonl', 'OptMATH_Bench_166.jsonl','OptiBench.jsonl']
for filepath in testdataset:
    
    # loading data
    print('Loading data', filepath)
    test_data = [i for i in load_jsonl(os.path.join(datapath, filepath)) for _ in range(8)]
    print('Finish Loading')
    
    # generation 
    
    prompt_list = []
    for item in test_data:
        prompt_list.append(mp_worker(item))
    result_strs = generate_with_model(model, prompt_list, sampling_params)
    snippet_package_cor = []
    score = []
    snippet_package_tmp=[]
    # check the pass@8 accuracy
    
    result_chunks = [result_strs[i:i + 8] for i in range(0, len(result_strs), 8)]
    test_data_chunks = [test_data[i:i + 8] for i in range(0, len(test_data), 8)]
    for result_chunk, items in zip(result_chunks,test_data_chunks):
        for chunk, item in zip(result_chunk, items):
            snippet_package_tmp.append(check_result(chunk, item, solver_name))
        if 1 in snippet_package_tmp:
            snippet_package_cor.append(1)
        else:
            snippet_package_cor.append(0)
        snippet_package_tmp.clear()
    result = np.bincount(snippet_package_cor)
    print(f'Numbers of test cases in dataset {filepath}: {sum(result)}')
    print(f'Numbers of pass@8 cases in dataset {filepath}: {result[1]}')
    print(f'pass@8 accuracy for dataset {filepath}: {result[1]}/{sum(result)} = {result[1] / sum(result)}')
    print('-------------------------------------------------------------------')
    


Loading data NL4OPT.jsonl
Finish Loading


Processed prompts: 100%|██████████| 1960/1960 [03:06<00:00, 10.50it/s, est. speed input: 3428.87 toks/s, output: 7892.34 toks/s]


Numbers of test cases in dataset NL4OPT.jsonl: 245
Numbers of pass@8 cases in dataset NL4OPT.jsonl: 238
pass@8 accuracy for dataset NL4OPT.jsonl: 238/245 = 0.9714285714285714
-------------------------------------------------------------------
Loading data MAMO_EasyLP.json
Finish Loading


Processed prompts: 100%|██████████| 5216/5216 [09:39<00:00,  9.01it/s, est. speed input: 3822.21 toks/s, output: 7719.13 toks/s]


Numbers of test cases in dataset MAMO_EasyLP.json: 652
Numbers of pass@8 cases in dataset MAMO_EasyLP.json: 589
pass@8 accuracy for dataset MAMO_EasyLP.json: 589/652 = 0.9033742331288344
-------------------------------------------------------------------
Loading data MAMO_ComplexLP.json
Finish Loading


...

Numbers of test cases in dataset OptMATH_Bench_193.jsonl: 193
Numbers of pass@8 cases in dataset OptMATH_Bench_193.jsonl: 68
pass@8 accuracy for dataset OptMATH_Bench_193.jsonl: 68/193 = 0.35233160621761656
-------------------------------------------------------------------
Loading data OptMATH_Bench_166.jsonl
Finish Loading


Processed prompts: 100%|██████████| 1328/1328 [07:53<00:00,  2.81it/s, est. speed input: 3330.11 toks/s, output: 4967.84 toks/s]


Numbers of test cases in dataset OptMATH_Bench_166.jsonl: 166
Numbers of pass@8 cases in dataset OptMATH_Bench_166.jsonl: 46
pass@8 accuracy for dataset OptMATH_Bench_166.jsonl: 46/166 = 0.27710843373493976
-------------------------------------------------------------------
Loading data OptiBench.jsonl
Finish Loading


Processed prompts: 100%|██████████| 4840/4840 [08:32<00:00,  9.44it/s, est. speed input: 3435.35 toks/s, output: 7643.26 toks/s]


Numbers of test cases in dataset OptiBench.jsonl: 605
Numbers of pass@8 cases in dataset OptiBench.jsonl: 379
pass@8 accuracy for dataset OptiBench.jsonl: 379/605 = 0.6264462809917355
-------------------------------------------------------------------
